# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：

In [1]:
pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 127.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [3]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model"
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit"

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [4]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [5]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [6]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
print(response[0].split("### Response:")[1])


<think>
好的，我来仔细分析一下这个问题。首先，用户是一名28岁的男性程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时还恶心。这是一些常见的症状，需要结合临床推理、诊断学和治疗规划来综合分析。

首先，头晕和脖子疼可能与神经系统的某些问题有关，比如脑力损伤、神经系统疾病或者运动神经损伤。同时，恶心也有可能是由于消化系统的问题，比如胃炎、食管炎或者胃食管反流病。不过，由于用户是程序员，可能有工作压力较大，导致这些症状。

接下来，考虑可能的诊断因素。头晕和脖子疼可能与脑力损伤有关，如中风、脑力损伤综合征等。恶心可能与消化系统疾病相关，比如胃炎、食管炎或胃食管反流病。但考虑到用户是程序员，可能在工作压力下，这些症状可能不是由脑力损伤直接引起的，而是由长期的神经损伤或运动神经损伤导致的。

另外，可能需要考虑是否存在神经系统疾病，如中风，或者运动神经损伤。由于用户是程序员，可能有脑力损伤，但需要进一步评估。此外，消化系统疾病如胃炎也可能需要排除。

治疗方面，可能需要综合治疗，包括急性治疗（如药物治疗，如抗恶心药、抗抑郁药）和慢性治疗（如物理治疗，如平衡训练）。此外，心理治疗可能也是必要的，因为长期的工作压力可能会影响患者的神经功能。

综合来看，可能的诊断包括中风、脑力损伤综合征、神经损伤（如运动神经损伤）或消化系统疾病。治疗方面，可能需要药物治疗、物理治疗和心理治疗。
</think>

### 分步思考过程：

1. **问题分析**：
   - 用户为男性，28岁，程序员，工作持续到半夜，导致头晕、脖子疼和恶心。
   - 问题涉及神经系统的常见症状，可能与神经损伤相关。

2. **可能的诊断因素**：
   - **头晕和脖子疼**：可能是神经系统的损伤，如中风、脑力损伤或运动神经损伤。
   - **恶心**：可能与消化系统疾病（如胃炎、食管炎）或神经系统的其他问题相关。

3. **临床推理**：
   - 程序员可能长期处于高压工作环境中，可能引发脑力损伤或运动神经损伤。
   - 潜在的神经损伤可能导致头晕、脖子疼和恶心。

4. **诊断建议**：
   - **脑力损伤**：考虑中风、脑力损伤综合征等。
   - **神经损伤**：考虑运动神经损伤、脑部损伤或神经中枢问题。
   - **消化系统疾病**：考虑胃炎、食管炎或反流性胃炎

---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh


In [8]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [9]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20171 [00:00<?, ? examples/s]

Map:   0%|          | 0/20171 [00:00<?, ? examples/s]

In [10]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [11]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [12]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536, padding_idx=151654)
        (layers): ModuleList(
          (0): Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
       

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [13]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [14]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 18,464,768 of 1,795,552,768 (1.03% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.160600
2,3.169000
3,3.077400
4,3.165800
5,3.149700
6,3.008400
7,2.984600
8,2.946500
9,2.901200
10,2.794600


TrainOutput(global_step=60, training_loss=2.4793787558873492, metrics={'train_runtime': 519.3894, 'train_samples_per_second': 14.787, 'train_steps_per_second': 0.116, 'total_flos': 7.163728311484416e+16, 'train_loss': 2.4793787558873492})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [15]:
model.save_pretrained("qwen-1.5b_lora_model")

In [16]:
tokenizer.save_pretrained("qwen-1.5b_lora_model")

('qwen-1.5b_lora_model/tokenizer_config.json',
 'qwen-1.5b_lora_model/special_tokens_map.json',
 'qwen-1.5b_lora_model/chat_template.jinja',
 'qwen-1.5b_lora_model/tokenizer.json')

### 9. 测试训练后的生成结果

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [18]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人，5天已经得了急性阑尾炎，还有一点腹痛和发热，但还是有右下腹的压痛包块。这个压痛包块，看起来很严重，可能就是阑尾炎的典型表现。

首先，这个压痛包块是阑尾炎的常见症状之一，它通常来自阑尾部的炎症反应，尤其是急性阶段。所以，根据这个情况，需要立即处理。

急性阑尾炎的患者应该立即就医，因为症状已经很严重，这会带来严重的后果。因此，立即进行紧急处理是必要的。

立即的紧急处理包括立即就医，这能帮助医生快速诊断和治疗。然而，患者在紧急情况下可能需要立即停止用药，尤其是抗生素，以避免进一步的感染。

根据这个情况，患者应该立即停止抗生素治疗。这可能有助于减轻感染，防止进一步扩散。同时，观察压痛包块是否还有其他症状，比如疼痛或肿胀，如果症状没有改善，可能需要进一步的治疗。

在紧急情况下，及时处理是非常重要的。因此，患者需要立即就医，并立即停止抗生素使用。

综上所述，立即就医是治疗急性阑尾炎的必要步骤。患者应该立即停止抗生素，并观察症状，以便获得进一步的诊断和治疗。
</think>
根据患者的症状描述，急性阑尾炎的患者已经发病5天，腹痛稍有减轻但仍然发热，并且在体检时发现右下腹有压痛的包块。在这种情况下，立即的紧急处理是必要的。

急性阑尾炎通常表现为压痛包块，这通常与阑尾炎的急性阶段有关。因此，患者需要立即就医以获得准确的诊断和治疗。

为了防止进一步的感染，患者应该立即停止使用抗生素。由于压痛包块仍然存在，建议立即观察症状，以便获得进一步的诊断。

因此，患者应立即就医，并立即停止抗生素治疗。这有助于减轻感染，并在进一步诊断和治疗中提供有效的治疗方案。



In [19]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )

    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [20]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这位60岁的男患者，表现右侧胸痛，X线检查显示右侧肋膈角消失，这很像是肺结核的症状。那肺结核是怎么回事呢？我记得肺结核是肺部的结核病，通常会导致胸腔积液。所以，这个患者可能有肺结核。

现在，问题来了，这位患者出现的是肺结核，对吗？所以，肺结核通常会导致肺部的结核病，导致胸腔积液。那问题是要找一个实验室检查，来了解这个胸腔积液的性质。

嗯，首先，我想到了X线检查已经显示了肋膈角消失，所以这可能是一个肺结核的表现。那么，另一个可能性是，这个胸腔积液可能来自结核病。

那么，接下来，我需要考虑一些可能的实验室检查，来看看哪种方法能帮助了解这个胸腔积液的性质。

首先，血常规检查可以告诉我血浆蛋白的水平，这可以帮助了解是否有感染性或感染性变化，但其实，对于肺结核这种疾病，血常规可能对肺部的感染性表现不一定有直接帮助。

然后是胸腔穿刺检查，这是常用的检查方法。它可以帮助我们了解胸腔的液体状态，比如是否有液体积液。但其实，对于肺结核，胸腔积液是由肺结核引起的，所以这可能能直接帮助我们了解这一点。

再考虑一下尿常规，它可以帮助了解是否有尿液中的细菌或病毒，但这可能对肺结核的诊断没有直接的帮助，因为肺结核通常不会引起尿道感染。

至于体格检查，比如呼吸系统检查，它可以帮助了解患者的呼吸状况，比如是否出现呼吸困难，这可能对于肺结核患者也有一定的帮助。

然后是心肺功能检查，比如心肺电图，这可能有助于了解心肺功能，但这可能不是最直接的手段来了解胸腔积液。

再想一下，胸腔穿刺检查，它确实可以帮助了解胸腔液体的状态，比如液体是否是液体性的还是气体性的。对于肺结核，肺部的液体可能来自肺泡，所以胸腔穿刺检查可以帮助我们了解这一点。

嗯，所以，我想到，胸腔穿刺检查是了解肺腔液体状态的最佳方法。它能明确地告诉我们，这个胸腔液是液体性的还是气体性的，这对于肺结核的诊断和治疗都很重要。

嗯，所以，综合来看，胸腔穿刺检查对了解胸腔积液的性质是有帮助的。
</think>
胸腔穿刺检查对于了解肺腔液体状态非常有用。肺结核通常是由于肺部感染导致的，而肺部的液体通常来自于肺泡，所以胸腔穿刺检查可以帮助明确知道这些液体是液体性的还是气体性的。这对于诊断和治疗肺结核患者至关重要，因为了解

In [21]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这个28岁的男性患者，工作是程序员，长期熬夜，最近突然头晕目眩，甚至有点恶心，这让我想到可能有高血压的问题。高血压的话，通常会有高血压引起的头晕目眩，尤其是夜间。不过，也有可能是其他原因造成的。再想想，这个患者工作很累，长期熬夜，可能还与睡眠不足有关。睡眠不足会导致头晕和恶心。还有一种可能性是糖尿病，糖尿病患者常常有头晕，尤其是夜间。不过，糖尿病患者需要特别注意饮食和运动，避免熬夜。

然后，再看看高血压引起的晕眩，通常有高血压倾向。高血压患者往往会有头晕、恶心、头痛和口干舌燥等症状。这个患者有头晕和恶心，也有可能是高血压。不过，高血压患者还需要注意饮食和睡眠，避免熬夜。

再想一下，这个患者长期熬夜，可能是因为长期的疲劳，导致睡眠不足。如果他有高血压，可能需要调整睡眠时间，但同时也要注意饮食和运动，避免熬夜。

不过，也有可能他有糖尿病。糖尿病患者如果长期熬夜，可能会有头晕，但通常不会伴随恶心。不过，如果他有糖尿病，同时有高血压，可能需要特别注意饮食和运动，避免熬夜。

再看看，这个患者的工作是程序员，长期熬夜，可能还有其他原因，比如脑力损伤导致的脑血管疾病，比如高血压或者糖尿病。或者，他可能有慢性肾病，因为长期的睡眠不足可能导致肾功能不佳。

综合来看，最有可能的是高血压或者糖尿病。因为高血压和糖尿病都可能导致头晕目眩和恶心，同时，长期熬夜可能与睡眠不足有关。此外，糖尿病患者需要特别注意饮食和运动，避免熬夜。

不过，也有可能是心脑血管疾病，比如心脑血管疾病，但这种情况下，患者通常会有心慌、呼吸困难和意识模糊，而不是头晕目眩和恶心。

再想想，高血压患者需要考虑饮食和睡眠，避免熬夜。同时，糖尿病患者需要特别注意饮食和运动，避免熬夜。

综合考虑，最有可能是高血压或糖尿病。因为这两种疾病通常与长期的睡眠不足有关，导致头晕目眩和恶心。而心脑血管疾病通常与心力衰竭有关，患者会有心慌等症状，而不是头晕目眩和恶心。

再看下，患者工作是程序员，长期熬夜，这可能与脑力损伤有关，导致心力衰竭。心脑血管疾病包括高血压、糖尿病、肾病、心脏病等，这些疾病通常与心力衰竭有关，患者的头晕目眩和恶心可能与这些疾病有关。

不过，还有一种可能是高血压和糖尿病同时存在，但需要特别注意

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [23]:
!cd /content && zip -r qwen-1.5b_lora_model.zip qwen-1.5b_lora_model

  adding: qwen-1.5b_lora_model/ (stored 0%)
  adding: qwen-1.5b_lora_model/adapter_config.json (deflated 57%)
  adding: qwen-1.5b_lora_model/special_tokens_map.json (deflated 70%)
  adding: qwen-1.5b_lora_model/chat_template.jinja (deflated 75%)
  adding: qwen-1.5b_lora_model/tokenizer_config.json (deflated 88%)
  adding: qwen-1.5b_lora_model/tokenizer.json (deflated 81%)
  adding: qwen-1.5b_lora_model/README.md (deflated 65%)
  adding: qwen-1.5b_lora_model/adapter_model.safetensors (deflated 8%)
